In [ ]:
import requests
import string
import datetime
import time

import pandas as pd

from bs4 import BeautifulSoup

In [ ]:
domains = []

for letter_1 in string.ascii_lowercase:
    for letter_2 in string.ascii_lowercase:
        domains.append(letter_1 + letter_2 + ".com")
        
        for letter_3 in string.ascii_lowercase:
            domains.append(letter_1 + letter_2 + letter_3 + ".com")

In [ ]:
%%time

domain_information = {}

headers =    {"Host": "reports.internic.net",
              "User-Agent": "Mozilla/5.0 (Windows NT 10.0; WOW64; rv:61.0) Gecko/20100101 Firefox/61.0",
              "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
              "Accept-Language": "en-US,en;q=0.5",
              "Referer": "https://www.internic.net/",
              "DNT": "1",
              "Connection": "keep-alive",
              "Upgrade-Insecure-Requests": "1"}

domains_done = 0
for domain_name in domains:
    params = {"whois_nic": domain_name, 
              "type": "domain"}
    
    info = 0
    
    # Sometimes an empty webpage is returned
    while len(info) < 100:
        time.sleep(5)
        
        r = requests.get("https://reports.internic.net/cgi/whois", 
                 params = params,
                 headers = headers)
        
        soup = BeautifulSoup(r.text, "html.parser")
        info = str(soup.findAll('pre'))
    
    creation_date = info[(info.find('Creation Date: ') + len('Creation Date: ')):info.find('\r\n   Registry Expiry Date:')]
    expiration_date = info[(info.find('Registry Expiry Date: ') + len('Registry Expiry Date: ')):info.find('\r\n   Registrar: ')]
    updated_date = info[(info.find('Updated Date: ') + len('Updated Date: ')):info.find('\r\n   Creation Date:')]
    
    # The Z denotes timezone UTC
    if creation_date[-1] == "Z":
        creation_date = creation_date[:-1]
    if expiration_date[-1] == "Z":
        expiration_date = expiration_date[:-1]
    if updated_date[-1] == "Z":
        updated_date = updated_date[:-1]
    
    try:
        creation_date = datetime.datetime.strptime(creation_date, '%Y-%m-%dT%H:%M:%S')
        expiration_date = datetime.datetime.strptime(expiration_date, '%Y-%m-%dT%H:%M:%S')
        updated_date = datetime.datetime.strptime(updated_date, '%Y-%m-%dT%H:%M:%S')
    except ValueError:
        print(domain_name)
    
    domain_information[domain_name] = {"Creation date": creation_date,
                                      "Experation date": expiration_date,
                                      "Update date": updated_date}
    
    domains_done += 1
    time.sleep(5)
    
    if domains_done % 100 == 0:
        print("A total of {} domains is scraped, at {:0.3f} %".format(domains_done, domains_done / len(domains) * 100))
        
        data = pd.DataFrame.from_dict(domain_information, orient = "index")
        data.to_csv("Data\Domain Information.csv")
    
    data.to_csv("Data\Domain Information.csv")

In [ ]:
data = data.sort_index()
data.to_csv("Data\Domain Information.csv")
data.head()